# Multi-Qubit Entropy Production Simulation

This notebook demonstrates the generalized L-qubit correlation matrix simulator for arbitrary chain lengths.

## System Description

**Hamiltonian:** $H = J \sum_{i=1}^{L-1} \sigma_i^x \sigma_{i+1}^x$ (nearest-neighbor XX coupling)

**Evolution equation:**
$$
|d\psi\rangle = -iH\,dt\,|\psi\rangle + \sum_{i=1}^{L}\left[\epsilon \xi_i \frac{\sigma_i^z - \langle\sigma_i^z\rangle}{2} - \frac{\epsilon^2}{2}\left(\frac{\sigma_i^z - \langle\sigma_i^z\rangle}{2}\right)^2\right]|\psi\rangle
$$

**Initial state:** $|\uparrow\uparrow...\uparrow\rangle$ (all spins up)

**Entropy production:**
$$
Q = \sum_{k=1}^{L} \sum_{i=1}^{N} \left[2\epsilon^2 z_{i-1,k}\frac{z_{i-1,k}+z_{i,k}}{2} + 2\epsilon\xi_{i,k}\frac{z_{i-1,k}+z_{i,k}}{2}\right]
$$

**Theoretical prediction** (for $J \gg 1/\tau$):
$$
\langle Q \rangle = L \times \frac{T}{\tau}(A+1)
$$

where $T/\tau = N\epsilon^2$ and $A = \langle z^2\rangle$. If $A = 1/2$, then $\langle Q\rangle = \frac{3L}{2} \cdot \frac{T}{\tau}$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from multi_qubit_correlation_simulator import MultiQubitCorrelationSimulator
import sys
sys.path.append('..')

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

## 1. Single Trajectory Visualization for L=4 Qubits

In [ ]:
# Create simulator with L=4 qubits
L = 4
sim = MultiQubitCorrelationSimulator(
    L=L,
    J=10.0,
    epsilon=0.01,
    N_steps=10000,
    T=1.0,
    periodic=False
)

print(f"Simulation parameters:")
print(f"  L = {sim.L} qubits")
print(f"  J = {sim.J}")
print(f"  ε = {sim.epsilon}")
print(f"  N = {sim.N_steps}")
print(f"  T = {sim.T}")
print(f"  dt = {sim.dt:.6f}")
print(f"  T/τ = N×ε² = {sim.N_steps * sim.epsilon**2:.3f}")
print(f"  Theoretical ⟨Q⟩ = (3L/2)×T/τ = {1.5 * L * sim.N_steps * sim.epsilon**2:.3f} (if A=0.5)")
print(f"  Boundary: {'Periodic' if sim.periodic else 'Open'}")

In [ ]:
# Simulate single trajectory
Q, z_traj, xi_traj = sim.simulate_trajectory()

print(f"\nSingle trajectory results:")
print(f"  Q = {Q:.3f}")
print(f"  Initial z = {z_traj[0]}")
print(f"  Final z = {z_traj[-1]}")

# Plot magnetization dynamics for all qubits
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

times = np.linspace(0, sim.T, sim.N_steps + 1)
colors = plt.cm.viridis(np.linspace(0, 1, L))

# Plot all magnetizations
for i in range(L):
    axes[0, 0].plot(times, z_traj[:, i], label=f'z_{i+1}', alpha=0.7, color=colors[i])
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('<σ_z>')
axes[0, 0].set_title('Magnetization vs Time (All Qubits)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot squared magnetizations
for i in range(L):
    axes[0, 1].plot(times, z_traj[:, i]**2, label=f'z_{i+1}^2', alpha=0.7, color=colors[i])
axes[0, 1].axhline(y=0.5, color='r', linestyle='--', label='A=0.5', alpha=0.5)
axes[0, 1].set_xlabel('Time')
axes[0, 1].set_ylabel('<σ_z>^2')
axes[0, 1].set_title('Squared Magnetization')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot average magnetization
z_avg = np.mean(z_traj, axis=1)
axes[1, 0].plot(times, z_avg, label='Average', color='black', linewidth=2)
axes[1, 0].fill_between(times, 
                         np.min(z_traj, axis=1), 
                         np.max(z_traj, axis=1), 
                         alpha=0.3, label='Min-Max range')
axes[1, 0].set_xlabel('Time')
axes[1, 0].set_ylabel('⟨z⟩')
axes[1, 0].set_title('Average Magnetization Across Chain')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Heatmap of magnetization over time and space
im = axes[1, 1].imshow(z_traj.T, aspect='auto', cmap='RdBu_r', 
                        extent=[0, sim.T, L-0.5, -0.5], vmin=-1, vmax=1)
axes[1, 1].set_xlabel('Time')
axes[1, 1].set_ylabel('Qubit index')
axes[1, 1].set_title('Magnetization Heatmap')
axes[1, 1].set_yticks(range(L))
axes[1, 1].set_yticklabels([f'{i+1}' for i in range(L)])
plt.colorbar(im, ax=axes[1, 1], label='⟨σ_z⟩')

plt.tight_layout()
plt.show()

## 2. Ensemble Simulation and Comparison with Theory

In [ ]:
# Run ensemble simulation
n_trajectories = 5000
print(f"Running ensemble simulation with {n_trajectories} trajectories for L={L} qubits...")

Q_values, z_trajectories, xi_trajectories = sim.simulate_ensemble(
    n_trajectories=n_trajectories,
    progress=True
)

print("\nEnsemble simulation complete!")

In [ ]:
# Calculate statistics
mean_Q = np.mean(Q_values)
std_Q = np.std(Q_values)
variance_Q = np.var(Q_values)
theoretical_Q = sim.theoretical_prediction()

# Calculate observed A = ⟨z²⟩
z_squared = z_trajectories**2  # shape (n_traj, N_steps+1, L)
A_per_qubit = np.mean(z_squared, axis=(0, 1))  # Average over trajectories and time
A_total = np.mean(z_squared)  # Overall average

# Calculate predicted Q using observed A
T_over_tau = sim.N_steps * sim.epsilon**2
predicted_Q_from_A = sim.L * T_over_tau * (A_total + 1.0)

print(f"\n{'='*60}")
print(f"ENSEMBLE RESULTS (n = {n_trajectories}, L = {L})")
print(f"{'='*60}")
print(f"\nEntropy Production:")
print(f"  Observed ⟨Q⟩ = {mean_Q:.4f} ± {std_Q:.4f} (variance = {variance_Q:.4f})")
print(f"  Theoretical ⟨Q⟩ (A=0.5) = {theoretical_Q:.4f}")
print(f"  Predicted (observed A) = {predicted_Q_from_A:.4f}")
print(f"  Ratio (obs/theory) = {mean_Q/theoretical_Q:.4f}")
print(f"\nMagnetization Squared:")
for i in range(L):
    print(f"  A_{i+1} = ⟨z_{i+1}²⟩ = {A_per_qubit[i]:.4f}")
print(f"  A_avg = {A_total:.4f}")
print(f"  Expected A = 0.5")
print(f"\nParameters:")
print(f"  L = {sim.L}")
print(f"  T/τ = {T_over_tau:.4f}")
print(f"  J = {sim.J}")
print(f"  ε = {sim.epsilon}")

## 3. Distribution of Q

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

# Histogram of Q
ax.hist(Q_values, bins=50, density=True, alpha=0.7, edgecolor='black')
ax.axvline(mean_Q, color='r', linestyle='--', linewidth=2, label=f'Observed mean = {mean_Q:.3f}')
ax.axvline(theoretical_Q, color='g', linestyle='--', linewidth=2, label=f'Theory (A=0.5) = {theoretical_Q:.3f}')
ax.set_xlabel('Q')
ax.set_ylabel('Probability Density')
ax.set_title(f'Distribution of Entropy Production Q (L={L})')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Time-Averaged Magnetization Statistics

In [ ]:
# Compute time-averaged z values for each trajectory
z_time_avg = np.mean(z_trajectories, axis=1)  # shape (n_traj, L)
z_squared_time_avg = np.mean(z_trajectories**2, axis=1)  # shape (n_traj, L)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Distribution of time-averaged magnetization (all qubits)
for i in range(L):
    axes[0, 0].hist(z_time_avg[:, i], bins=40, density=True, alpha=0.5, 
                    label=f'z_{i+1}', color=colors[i])
axes[0, 0].axvline(0, color='k', linestyle='--', linewidth=1, alpha=0.5)
axes[0, 0].set_xlabel('Time-averaged ⟨z⟩')
axes[0, 0].set_ylabel('Probability Density')
axes[0, 0].set_title('Distribution of ⟨z_i⟩_time')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Distribution of time-averaged z² (all qubits)
for i in range(L):
    axes[0, 1].hist(z_squared_time_avg[:, i], bins=40, density=True, alpha=0.5,
                    label=f'z_{i+1}²', color=colors[i])
axes[0, 1].axvline(0.5, color='r', linestyle='--', linewidth=2, label='Theory A=0.5')
axes[0, 1].set_xlabel('Time-averaged ⟨z²⟩')
axes[0, 1].set_ylabel('Probability Density')
axes[0, 1].set_title('Distribution of ⟨z_i²⟩_time')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Mean A per qubit across ensemble
axes[1, 0].bar(range(1, L+1), A_per_qubit, color=colors, alpha=0.7, edgecolor='black')
axes[1, 0].axhline(0.5, color='r', linestyle='--', linewidth=2, label='Theory A=0.5')
axes[1, 0].set_xlabel('Qubit index')
axes[1, 0].set_ylabel('⟨z²⟩')
axes[1, 0].set_title('Average A per Qubit')
axes[1, 0].set_xticks(range(1, L+1))
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Correlation between qubits (time-averaged)
z_avg_per_traj = np.mean(z_time_avg, axis=1)  # Average over all qubits
axes[1, 1].hist(z_avg_per_traj, bins=40, density=True, alpha=0.7, edgecolor='black')
axes[1, 1].axvline(np.mean(z_avg_per_traj), color='r', linestyle='--', linewidth=2,
                   label=f'Mean = {np.mean(z_avg_per_traj):.3f}')
axes[1, 1].set_xlabel('Chain-averaged ⟨z⟩')
axes[1, 1].set_ylabel('Probability Density')
axes[1, 1].set_title('Distribution of Chain-Averaged Magnetization')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Scaling with Chain Length L

In [ ]:
# Scan over different L values
L_values = [2, 3, 4, 5, 6]
n_traj_scan = 1000
epsilon_fixed = 0.1
N_steps_fixed = 1000
J_fixed = 10.0

results_L = []

print("Scanning over L values...")
for L_val in L_values:
    sim_scan = MultiQubitCorrelationSimulator(
        L=L_val,
        J=J_fixed,
        epsilon=epsilon_fixed,
        N_steps=N_steps_fixed,
        T=1.0,
        periodic=False
    )
    
    Q_vals, z_trajs, _ = sim_scan.simulate_ensemble(n_traj_scan, progress=False)
    
    mean_Q = np.mean(Q_vals)
    std_Q = np.std(Q_vals)
    A_obs = np.mean(z_trajs**2)
    theory_Q = sim_scan.theoretical_prediction()
    
    results_L.append({
        'L': L_val,
        'mean_Q': mean_Q,
        'std_Q': std_Q,
        'A': A_obs,
        'theory_Q': theory_Q
    })
    print(f"  L={L_val}: ⟨Q⟩={mean_Q:7.3f}, Theory={theory_Q:7.3f}, Ratio={mean_Q/theory_Q:.3f}, A={A_obs:.4f}")

# Extract data
L_array = np.array([r['L'] for r in results_L])
mean_Q_array = np.array([r['mean_Q'] for r in results_L])
std_Q_array = np.array([r['std_Q'] for r in results_L])
A_array = np.array([r['A'] for r in results_L])
theory_Q_array = np.array([r['theory_Q'] for r in results_L])

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot mean Q vs L
axes[0].errorbar(L_array, mean_Q_array, yerr=std_Q_array/np.sqrt(n_traj_scan), 
                 fmt='o-', capsize=5, label='Observed', linewidth=2, markersize=8)
axes[0].plot(L_array, theory_Q_array, 's--', color='r', linewidth=2, 
             markersize=8, label='Theory (A=0.5)')
axes[0].set_xlabel('Chain Length L')
axes[0].set_ylabel('⟨Q⟩')
axes[0].set_title('Mean Entropy Production vs L')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot A vs L
axes[1].plot(L_array, A_array, 'o-', label='Observed A', linewidth=2, markersize=8)
axes[1].axhline(0.5, color='r', linestyle='--', linewidth=2, label='Theory A=0.5')
axes[1].set_xlabel('Chain Length L')
axes[1].set_ylabel('A = ⟨z²⟩')
axes[1].set_title('Average Squared Magnetization vs L')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot ratio (observed/theory) vs L
ratio_array = mean_Q_array / theory_Q_array
axes[2].plot(L_array, ratio_array, 'o-', linewidth=2, markersize=8)
axes[2].axhline(1.0, color='r', linestyle='--', linewidth=2, label='Perfect agreement')
axes[2].set_xlabel('Chain Length L')
axes[2].set_ylabel('⟨Q⟩_obs / ⟨Q⟩_theory')
axes[2].set_title('Agreement Ratio vs L')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nLinear scaling validation:")
print(f"  Theory predicts ⟨Q⟩ ∝ L")
print(f"  Observed scaling: ⟨Q⟩(L=6)/⟨Q⟩(L=2) = {mean_Q_array[-1]/mean_Q_array[0]:.3f}")
print(f"  Expected scaling: {L_values[-1]/L_values[0]:.3f}")

## 6. Comparison: Open vs Periodic Boundary Conditions

In [ ]:
# Compare open vs periodic boundary conditions for L=4
L_bc = 4
n_traj_bc = 2000

print(f"Comparing boundary conditions for L={L_bc}...")

# Open BC
sim_open = MultiQubitCorrelationSimulator(
    L=L_bc,
    J=10.0,
    epsilon=0.1,
    N_steps=1000,
    T=1.0,
    periodic=False
)
Q_open, z_open, _ = sim_open.simulate_ensemble(n_traj_bc, progress=True)

# Periodic BC
sim_periodic = MultiQubitCorrelationSimulator(
    L=L_bc,
    J=10.0,
    epsilon=0.1,
    N_steps=1000,
    T=1.0,
    periodic=True
)
Q_periodic, z_periodic, _ = sim_periodic.simulate_ensemble(n_traj_bc, progress=True)

# Calculate statistics
mean_Q_open = np.mean(Q_open)
mean_Q_periodic = np.mean(Q_periodic)
A_open = np.mean(z_open**2)
A_periodic = np.mean(z_periodic**2)

print(f"\nResults for L={L_bc}:")
print(f"  Open BC:     ⟨Q⟩ = {mean_Q_open:.3f}, A = {A_open:.4f}")
print(f"  Periodic BC: ⟨Q⟩ = {mean_Q_periodic:.3f}, A = {A_periodic:.4f}")
print(f"  Difference: {abs(mean_Q_periodic - mean_Q_open):.3f} ({100*abs(mean_Q_periodic - mean_Q_open)/mean_Q_open:.1f}%)")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Compare Q distributions
axes[0].hist(Q_open, bins=40, density=True, alpha=0.6, label='Open BC', edgecolor='black')
axes[0].hist(Q_periodic, bins=40, density=True, alpha=0.6, label='Periodic BC', edgecolor='black')
axes[0].axvline(mean_Q_open, color='blue', linestyle='--', linewidth=2)
axes[0].axvline(mean_Q_periodic, color='orange', linestyle='--', linewidth=2)
axes[0].set_xlabel('Q')
axes[0].set_ylabel('Probability Density')
axes[0].set_title(f'Entropy Production Distribution (L={L_bc})')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Compare A per qubit
A_per_qubit_open = np.mean(z_open**2, axis=(0, 1))
A_per_qubit_periodic = np.mean(z_periodic**2, axis=(0, 1))

x_pos = np.arange(1, L_bc+1)
width = 0.35
axes[1].bar(x_pos - width/2, A_per_qubit_open, width, label='Open BC', alpha=0.7, edgecolor='black')
axes[1].bar(x_pos + width/2, A_per_qubit_periodic, width, label='Periodic BC', alpha=0.7, edgecolor='black')
axes[1].axhline(0.5, color='r', linestyle='--', linewidth=2, label='Theory A=0.5')
axes[1].set_xlabel('Qubit index')
axes[1].set_ylabel('A = ⟨z²⟩')
axes[1].set_title('Average Squared Magnetization per Qubit')
axes[1].set_xticks(x_pos)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 7. Conclusions

This generalized multi-qubit simulator validates the theoretical prediction for arbitrary chain lengths:

$$
\langle Q \rangle = L \times \frac{T}{\tau}(A+1)
$$

### Key observations:

1. **Linear Scaling**: The entropy production scales linearly with chain length L, as expected from theory. Each qubit contributes independently to the total entropy production.

2. **Ergodic Limit**: In the strong coupling regime ($J \gg 1/\tau$), all qubits show $A \approx 0.5$, confirming ergodic behavior regardless of chain length.

3. **Boundary Effects**: For small L, open vs periodic boundary conditions show minimal difference in entropy production, suggesting that bulk properties dominate over edge effects.

4. **Spatial Uniformity**: The squared magnetization $A_i$ is approximately uniform across the chain, indicating no significant position-dependent effects in the ergodic regime.

5. **Numerical Stability**: The correlation matrix approach remains stable for longer chains (L > 2), demonstrating the robustness of the implementation.

### Future extensions:

- Long-range interactions beyond nearest-neighbor
- Different initial states (domain walls, alternating spins, etc.)
- Time-dependent coupling $J(t)$
- Site-dependent measurement strengths $\epsilon_i$
- Other Hamiltonian models (XY, XXZ, Heisenberg)